In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from fastai.tabular.all import *

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-sep-2021/train.csv" , nrows =5000)
test_df = pd.read_csv("../input/tabular-playground-series-sep-2021/test.csv")

In [ ]:
train_df['claim'].value_counts().plot.bar()
plt.show()


In [ ]:
train_df.dtypes

In [ ]:
splits = RandomSplitter()(range_of(train_df))
splits

In [ ]:
#cont_names  = train_df.drop(["id" , "claim"] ,axis=1).columns.tolist()
cont_names  = list(train_df.iloc[:,1:-1])
procs = [ Categorify ,FillMissing, Normalize]
y_names = 'claim'
y_block = CategoryBlock()

In [ ]:
to = TabularPandas(train_df, procs=procs,  cont_names=cont_names,
                   y_names=y_names, y_block=y_block, splits=splits)

In [ ]:
to.cont_names

In [ ]:
to.show()

In [ ]:
dls = to.dataloaders()

In [ ]:
#roc = RocAuc()
learn = tabular_learner(dls,  layers=[10, 10, 10], metrics=[accuracy, error_rate, Recall(), Precision()] )
#learn.fit(3, 1e-2 )


In [ ]:
learn.model

In [ ]:
learn.lr_find()

In [ ]:
learn.fit(5, 1e-1)

In [ ]:
learn.recorder.plot_loss()

In [ ]:
dl = learn.dls.test_dl(test_df)


In [ ]:
interpret = ClassificationInterpretation.from_learner(learn)
interpret.plot_confusion_matrix()

In [ ]:
preds = learn.get_preds(dl=dl)

In [ ]:
type(preds)

In [ ]:
preds

In [ ]:
final_preds = preds[0].numpy()
final_preds = np.argmax(final_preds, axis=1)

In [ ]:
final_preds

In [ ]:
submission_df = pd.read_csv( "../input/tabular-playground-series-sep-2021/sample_solution.csv")

In [ ]:
submission_df

In [ ]:
submission_df['claim'] = final_preds

In [ ]:
submission_df

In [ ]:
submission_df.to_csv('submission.csv', index=False)